<a href="https://colab.research.google.com/github/qcbegin/DSME6635-S24/blob/main/problem_sets/PS7_Unsupervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Set 7 - Unsupervised Learning (EM-Algorithm & Topic Modeling)

### Due at 12:30PM, Tuesday, April 23, 2024

Please first copy the CoLab file onto your own Google Drive. Finish the questions below and submit the **CoLab link** of your solutions in [this Google Sheet](https://docs.google.com/spreadsheets/d/1nOE-saTptG73WMCONDB1Z3pt-jHhmDA_1OHpQVHqQ1M/edit#gid=43704098). The total achievable points are 8 for this problem set. Please name you solution as

- `Member1LastName_Member1FirstName-Member2LastName_Member2FirstName_PS7.ipynb` (e.g., `Cao_Leo-Zhang_Renyu_PS7.ipynb`)



## 1. EM-Algorithm

The first exercise will be unsupervised learning, specifically EM algorithm. In particular, we will solve a very simple problem on mixture of gaussian.

    """
    Input:
        a list of numbers x_1, x_2, ..., x_n, suppose these numbers are drawn from two Gaussian distributions with variance 1 and mean mu_1 and mu_2
   
    Output:
        the probability of each number belongs to each Gaussian distribution.
    """

You will solve this problem using the famous [EM-Algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm).

The first function you will implement is called the **E-step**, which calculates each data point's probability belonging to each Gaussian distribution. Suppose you have gaussian distribution with mean $\mu_c$ and standard deviation $\sigma_c$, and $\pi_c$ is the probability that a point comes from distribution $c$. When you have points $x_i$, the E-step's equation is as follows:

$$ r_{ic} = \frac{\pi_c N(x_i | \mu_c, \sigma_c)}{\sum_{k=1}^K \pi_k N(x_i | \mu_k, \sigma_k)} $$

where

$$ N(x_i, \mu_c, \sigma_c) = \frac{1}{(2\pi)^{n/2}\sigma_c} exp\left(\frac{(x_i-\mu_c)^2\sigma_c^2}{2}\right).$$

In our case, the number of gaussian distribution is 2, so K = 2. To make your life easier, you can use scipy.stats.norm to compute the probability of a Gaussian distribution.




In [1]:
import numpy as np
from scipy.stats import norm

def e_step(data_points, mu_1, sigma_1, mu_2, sigma_2, pi=np.array([1/2, 1/2])):
    """
    This function conducts the e-step for gaussian mixture model with 2 distributions.
    Input:
        data_points: the data points from two gaussian.
        mu_1, sigma_, mu_2, sigma_2: the prior parameters for the two gaussian.
        pi: the total weights for each gaussian
    Output:
        probs: a matrix representing each point's probabiltiies towards each gaussian.
        probs[:, 0] is the probability of each point towards the first gaussian.
    """
    probs = np.zeros((len(data_points), 2))
    pi = np.array([1/2,1/2])

    ### BEGIN SOLUTION
    probs[:, 0] = pi[0] * norm(mu_1, sigma_1).pdf(data_points)
    probs[:, 1] = pi[1] * norm(mu_2, sigma_2).pdf(data_points)
    prob_sum = np.sum(probs, axis=1)
    probs = probs / prob_sum[:, None]
    ### END SOLUTION

    return probs


In [2]:
data_points = [0.1, 0.3, 10, 3]
probs = e_step(data_points, 0, 1, 0, 1)
assert np.array_equal(probs, np.ones((4, 2)) * 0.5)

The second function you will implement is the **M-step**. In particular, you will use the probabilities that each data point belongs to each distribution to update these distributions mean and standard deviations as the weighted averages of these data points (weighted by the probabilites).

Suppose that $m_c$ is the total weights towards a distribution $c$, then you should compute based on following equations:
$$ m_c = \sum r_{ic}$$
$$ \pi_c = \frac{m_c}{N}$$
$$ \mu_c = \frac{1}{m_c}\sum_i r_{ic} x_i $$
$$ \sigma_c^2 = \frac{1}{m_c}\sum_i r_{ic} (x_i - \mu_c)^2 $$

In [3]:
def m_step(data_points, probs):
    """
    This function performs the M-step for Gaussian Mixture Model.
    Input:
        data_points: a list of data points
        probs: a matrix representing each data point's probabilties to the 2 distributions.
    Output:
        pi: the probability of each distribution in all data points.
        mu_1, sigma_1, mu_2, sigma_2: estimated parameters for each distribution
    """
    pi = mu_1 = mu_2 = sigma_1 = sigma_2 = 0

    ### BEGIN SOLUTION
    mu_1 = np.dot(probs[:, 0], data_points) / np.sum(probs[:, 0])
    mu_2 = np.dot(probs[:, 1], data_points) / np.sum(probs[:, 1])
    
    sigma_1 = np.sqrt(np.dot(probs[:, 0], (data_points - mu_1) ** 2) / np.sum(probs[:, 0]))
    sigma_2 = np.sqrt(np.dot(probs[:, 1], (data_points - mu_2) ** 2) / np.sum(probs[:, 1]))

    pi = np.sum(probs, axis=0) / len(data_points)

    ### END SOLUTION

    return pi, mu_1, sigma_1, mu_2, sigma_2

In [4]:
data_points = [0.1, 0.3, 10, 3]
probs = e_step(data_points, 0, 1, 10, 5)
pi, mu_1, sigma_1, mu_2, sigma_2 = m_step(data_points, probs)
assert np.isclose(pi[0], 0.51762712)
assert np.isclose(mu_1, 0.3741766759682093)

Finally, combining the E-step and M-step, you will implement your EM-Algorithm that iteratively apply E and M steps until the probability distribution vector converges. You will return the final probability distribution as well as the mean and standard deviations of the two Gaussian random variables.

In particular, your code should work as follows:

1. Initialize two random Gaussians (may or may not be based on the data).

2. Iterativelly apply EM-Algorithm until the L1-norm of probability distribution difference is smaller than the tolerance.

3. Classify each point to one of the distirbution based on its probability distribution (which one is greater than 0.5).


In [5]:
def mixture_gaussian(data_points, tolerance = 0.00001):
    """
    This function performs the overall gaussian mixture models.
    Input:
        data_points: a list of data poitns.
        tolerance: the tolerance in stopping the EM algorithm.
    Output:
        pi: the probability of each distribution in all data points.
        mu_1, sigma_1, mu_2, sigma_2: estimated parameters for each distribution
    """
    pi = mu_1 = mu_2 = sigma_1 = sigma_2 = 0

    ### BEGIN SOLUTION
    # initialize the parameters
    mu_1, mu_2 = np.random.choice(data_points, 2, replace=False)
    sigma_1, sigma_2 = np.std(data_points), np.std(data_points)
    probs_old = np.zeros((len(data_points), 2))

    # iterate until convergence
    while True:
        # calculate the initial posterior probabilities (E-step)
        probs = e_step(data_points, mu_1, sigma_1, mu_2, sigma_2)
        # maximize the likelihood (M-step)
        pi, mu_1, sigma_1, mu_2, sigma_2 = m_step(data_points, probs)
        # check the L1 norm of the difference between the new and old probabilities
        diff = np.linalg.norm(probs - probs_old, ord=1)
        if diff < tolerance:
            break

        probs_old = probs.copy()
    ### END SOLUTION

    return pi, mu_1, sigma_1, mu_2, sigma_2


In [6]:
mu_1, sigma_1, mu_2, sigma_2 = 0, 1, 100, 10
data_points= np.concatenate([np.random.normal(mu_1, sigma_1, 3000),
                             np.random.normal(mu_2, sigma_2, 3000)])
pi, mu_1_e, sigma_1_e, mu_2_e, sigma_2_e = mixture_gaussian(data_points)
assert abs(mu_1_e-mu_1) < 0.1
assert abs(mu_2_e-mu_2) < 1

## 2. Topic Modeling

Now you have seen a typical unsupervised learning algorithm (i.e., Gaussian Mixture Models), let us now focus on one of the most useful unsupervised learning algorithms, topic modeling. In this exercise, you will be using another popular NLP package in Python, [**gensim**](https://github.com/RaRe-Technologies/gensim), to conduct topic modeling.

You can follow the guide [**here**](https://www.machinelearningplus.com/nlp/gensim-tutorial/).

In order to complete this task, you need to do the following in principle:

1. Use gensim's API to download the dataset.
2. Pre-process the data by removing stopwords and stemmizing them.
3. Create a gensim dictionary to reprseent each word and change the data into a gensim corpus.
4. Apply the LDA model on corpus and the dictionary.
5. Return the topics.

Luckily, you don't need to do Step 1 to 3, since you have already done this in prior exercises with other packages (Flair and Keras). You can focus on Steps 4 and 5.

We will be using the [**NYT article data from Kaggle**](https://www.kaggle.com/datasets/tumanovalexander/nyt-articles-data?resource=download&select=df_2019.csv).

**Note: You will need to upload `df_2019.csv` into your Google Colab notebook.**

**Note: This exercise will take some time since you are performing LDA updates on 5000 sentences.**

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import gensim
from gensim import corpora
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
# We need wordnet for lemmatizer

# Pre-processing function
lemma = WordNetLemmatizer()
def preprocess(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in set(stopwords.words('english'))])
    punc_free = ''.join(ch for ch in stop_free if ch not in set(string.punctuation))
    normalized = [lemma.lemmatize(word) for word in punc_free.split()]
    return normalized

# Load the NYT dataset and preprocess the fist 5000 articles
nyt_df = pd.read_csv("../Data/df_2019.csv")
documents = nyt_df['sentence'].tolist()
doc_clean = [preprocess(doc) for doc in documents[0:5000]]


def topic_modeling(doc_clean, num_topics=7):
    """
    This function takes a list of documents and return a gensim LDA model.
    Input:
        doc_clean: a list of documents.
        num_topics: number of topics in the docuemnt.
    Oputput:
        ldamodel: the LDA model from gensim based on doc_clean
    """
    # build the word to number dictionary
    dictionary = corpora.Dictionary(doc_clean)

    # change each document into the term matrix, we can use tfidf here if wanted
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    ldamodel = None

    ### BEGIN SOLUTION
    ldamodel = gensim.models.ldamodel.LdaModel(corpus=doc_term_matrix,
                                               id2word=dictionary,
                                               num_topics=num_topics,
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
    ### END SOLUTION

    return ldamodel

[nltk_data] Downloading package stopwords to /Users/Xinyu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Xinyu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/Xinyu/nltk_data...


In [8]:
ldamodel = topic_modeling(doc_clean, 10)
assert len(ldamodel.print_topics()) == 10
ldamodel.print_topics()

[(0,
  '0.050*"trump" + 0.047*"president" + 0.017*"leader" + 0.016*"wall" + 0.015*"right" + 0.015*"border" + 0.012*"house" + 0.012*"world" + 0.011*"school" + 0.011*"mr"'),
 (1,
  '0.065*"new" + 0.020*"york" + 0.015*"first" + 0.014*"book" + 0.013*"make" + 0.013*"may" + 0.012*"home" + 0.012*"city" + 0.011*"look" + 0.011*"back"'),
 (2,
  '0.030*"said" + 0.026*"say" + 0.022*"state" + 0.017*"woman" + 0.016*"american" + 0.012*"group" + 0.011*"united" + 0.011*"former" + 0.009*"many" + 0.008*"country"'),
 (3,
  '0.026*"work" + 0.017*"offer" + 0.017*"art" + 0.016*"deal" + 0.014*"number" + 0.012*"attack" + 0.012*"line" + 0.010*"harris" + 0.009*"theater" + 0.009*"dead"'),
 (4,
  '0.049*"shutdown" + 0.033*"government" + 0.015*"union" + 0.015*"history" + 0.015*"politics" + 0.015*"love" + 0.014*"case" + 0.014*"democratic" + 0.013*"address" + 0.012*"worker"'),
 (5,
  '0.025*"show" + 0.020*"would" + 0.017*"take" + 0.016*"like" + 0.014*"company" + 0.014*"open" + 0.013*"start" + 0.011*"season" + 0.010*"

# End of Problem Set 7.